In [2]:
import torch
import numpy
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import tensor, Tensor, nn

%matplotlib inline

In [3]:
class MultHeadAttention(nn.Module):
    def __init__(
        self,
        modelDim,
        headNum,
        dropout,
    ) -> None:
        """
        构建 多头 注意力机制 模块
        Args:
            modelDim:   词嵌入维度
            headNum:    单个模块里头的个数

        """

        assert modelDim % headNum == 0, "Can't segment the code"

        super().__init__()

        self.perDim = modelDim // headNum
        self.modelDim = modelDim
        self.headNum = headNum

        # 将多头的参数整合成统计给一个线性层, 增加并行度
        self.queayLinear = nn.Linear(modelDim, self.perDim * headNum)
        self.keyLinear = nn.Linear(modelDim, self.perDim * headNum)
        self.valueLinear = nn.Linear(modelDim, self.perDim * headNum)

        self.outputLinear = nn.Linear(modelDim, modelDim)

    def forward(self, key, query, value, attentionMask) -> Tensor:
        batchSize = key.size(0)
        res = query

        key = self.keyLinear(key)
        query = self.queayLinear(query)
        value = self.valueLinear(value)

        attentionOutput = key @ query.transpose(1, 2) / torch.sqrt(self.modelDim)

        attentionOutput = torch.softmax(attentionOutput, dim = 2)

        attentionOutput =  attentionOutput @ value

        return attentionOutput

In [4]:
class EncoderBlock(nn.Module):
    def __init__(
        self,
        dModel: int,
        dV: int,
    ) -> None:
        super(EncoderBlock, self).__init__()

In [16]:
a = torch.rand(64, 10, 20)
b = torch.rand(64, 10, 20)

c = a @ b.transpose(1, 2) / 10.0
c.shape

torch.Size([64, 10, 10])